In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (
    SparkSession.builder.appName("project1 preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executor.memory", "8g")
    .getOrCreate()
)

columns_wanted="ERP_P_38+ERP_P_37+ERP_P_36+ERP_P_35+ERP_P_34+ERP_P_33+ERP_P_32+ERP_P_31+ERP_P_30+ERP_P_29+ERP_P_28+ERP_P_27+ERP_P_26+ERP_P_25+ERP_P_24+ERP_P_23+ERP_P_22+ERP_P_21+ERP_P_19+ERP_P_18+ERP_P_17+ERP_P_16+ERP_P_15+ERP_P_14+ERP_P_13+ERP_P_12+ERP_P_11+ERP_P_10+ERP_P_8+ERP_P_9+ERP_P_7+ERP_P_6+ERP_P_5+ERP_P_4+ERP_P_3+ERP_P_2+EQUIV_2+EQUIV_10+EQUIV_9+EQUIV_8+EQUIV_7+EQUIV_6+EQUIV_5+EQUIV_4+EQUIV_3"

data_flag=columns_wanted.split("+")

your 131072x1 screen size is bogus. expect trouble
24/09/18 11:41:33 WARN Utils: Your hostname, Honor resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/18 11:41:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/18 11:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
def make_table(area_name, small_area_name):
    file_path=f"../data/landing/{area_name}_{small_area_name}.csv"
    output_path=f"../data/raw/{area_name}_{small_area_name}.csv"

    df=spark.read.csv(file_path, header=True, inferSchema=True)
    edited_df=df.groupBy("TIME_PERIOD") .pivot("MEASURE") .agg({"OBS_VALUE": "first"})
    edited_df.write.csv(output_path, header=True, mode="overwrite")

In [3]:
Boroondara_names=["ashburton(Vic.)", "Balwyn", "Balwyn_North", "Camberwell", "Glen_Iris_E", "Hawthorn_E", "Kew_East", "Surrey_Hills(W)"]

Manningham_West_names=["Bulleen", "Doncaster", "Templestowe", "Templestowe_Lower", "Doncaster_EN", "Doncaster_ES"]

Whitehorse_West_names=["Blackburn", "Blackburn_S", "Box_Hill", "Box_Hill_N", "Burwood(Vic)", "Burwood_E", "Surrey_Hills(E)"]

for small_area_name in Boroondara_names:
    make_table("Boroondara", small_area_name)

for small_area_name in Manningham_West_names:
    make_table("Manningham_West", small_area_name)

for small_area_name in Whitehorse_West_names:
    make_table("Whitehorse_West", small_area_name)


24/09/18 11:41:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
df=spark.read.csv("../data/raw/Manningham_West_Bulleen.csv")
print(df)

+-----------+--------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+-------+-------+-------+-------+-------+
|        _c0|     _c1|    _c2|    _c3|    _c4|    _c5|    _c6|    _c7|    _c8|    _c9|    _c10|    _c11|    _c12|    _c13|    _c14|    _c15|    _c16|    _c17|    _c18|    _c19|   _c20|    _c21|    _c22|    _c23|    _c24|    _c25|    _c26|    _c27|    _c28|    _c29|   _c30|    _c31|    _c32|    _c33|    _c34|    _c35|    _c36|    _c37|    _c38|    _c39|   _c40|   _c41|   _c42|   _c43|   _c44|   _c45|
+-----------+--------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+-

In [5]:
from urllib.request import urlretrieve

id="207011146"
columns_wanted="EQUIV_3"

url=f"https://api.data.abs.gov.au/data/ABS,ABS_REGIONAL_ASGS2021,/{columns_wanted}..{id}.A?startPeriod=2010&dimensionAtObservation=AllDimensions&format=csv"
output_file = f"../data/landing/test.csv"
urlretrieve(url, output_file)

file_path=f"../data/landing/test.csv"
output_path=f"../data/raw/test.csv"

df=spark.read.csv(file_path, header=True, inferSchema=True)
edited_df=df.groupBy("TIME_PERIOD") .pivot("MEASURE") .agg({"OBS_VALUE": "first"})
edited_df.write.csv(output_path, header=True, mode="overwrite")

24/09/18 19:05:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 9512519 ms exceeds timeout 120000 ms
24/09/18 19:05:13 WARN SparkContext: Killing executors is not supported by current scheduler.
24/09/18 19:09:46 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$